In [1]:
#imports
import numpy as np
import requests
import scipy.stats as st

# Option pricing model

In [2]:
# request the sofr rate
response = requests.get("https://markets.newyorkfed.org/api/rates/secured/sofr/last/1.json",)

# risk free rate
r = response.json()["refRates"][0]["percentRate"]/100

In [3]:
def blackScholes(sPrice, xPrice, sig, time, rate = r, type = "call"):
    d1 = (np.log(sPrice/xPrice) + (rate + (sig**2)/2) * time) / (sig*np.sqrt(time))
    d2 = d1 - sig*np.sqrt(time)
    try:
        if type == "call":
            price = sPrice*st.norm.cdf(d1) - xPrice*np.exp(-rate*time)*st.norm.cdf(d2)
            delta = st.norm.cdf(d1) # Measures impact of a change in the price of underlying
            theta = -(sPrice * st.norm.pdf(d1) * sig) / (2 * np.sqrt(time)) - rate * xPrice * np.exp(-rate * time) * st.norm.cdf(d2) # Measures impact of a change in time remaining      
        elif type == "put":
            price = xPrice*np.exp(-rate*time)*st.norm.cdf(-d2) - sPrice*st.norm.cdf(-d1)
            delta = -st.norm.cdf(-d1)
            theta = -(sPrice*st.norm.pdf(d1)*sig) / (2*np.sqrt(time)) + rate * xPrice * np.exp(-rate*time) * st.norm.cdf(-d2)
        gamma = st.norm.pdf(d1) / (sPrice*sig*np.sqrt(time)) # Measures the rate of change of delta
        vega = sPrice * np.sqrt(time) * st.norm.pdf(d1)
        return price, delta, gamma, theta/365, vega/100
    except:
        print("type can be either 'call' or 'put'")


In [4]:
blackScholes(15, 21, 0.36, 213/365, rate = 0.04)

(0.3046236019281898,
 0.15838560833942017,
 0.05859238680495298,
 -0.0025674614261444904,
 0.027695738343722024)